# Physics Validation Tutorial

This notebook guides you through validating the physics simulation parameters in the Digital Twin project.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import sys
import os

# Note: In a real environment, you would also import ROS 2 related libraries
# For this tutorial notebook, we'll simulate the physics validation

print("Physics Validation Tutorial")
print("=" * 30)
print("This notebook demonstrates how to validate physics parameters in the simulation")

## 1. Expected Physics Parameters

According to physics standards, we expect:
- Earth gravity: 9.81 m/s²
- Time for object to fall from height h: t = √(2h/g)
- Final velocity after falling from height h: v = √(2gh)

In [ ]:
# Define expected physics parameters
EXPECTED_GRAVITY = 9.81  # m/s^2
GRAVITY_TOLERANCE = 0.05  # m/s^2 (5% tolerance)

def calculate_fall_time(height, gravity=EXPECTED_GRAVITY):
    """Calculate time for object to fall from given height"""
    return math.sqrt(2 * height / abs(gravity))

def calculate_final_velocity(height, gravity=EXPECTED_GRAVITY):
    """Calculate velocity of object after falling from given height"""
    return math.sqrt(2 * abs(gravity) * height)

# Print expected values for common heights
heights = [1.0, 2.0, 5.0, 10.0]  # meters

print(f"Expected gravity: {EXPECTED_GRAVITY} m/s²")
print(f"Tolerance: ±{GRAVITY_TOLERANCE} m/s²")
print("\nExpected fall times and velocities:")
for h in heights:
    t = calculate_fall_time(h)
    v = calculate_final_velocity(h)
    print(f"  Height: {h:4.1f}m -> Time: {t:5.3f}s, Velocity: {v:5.2f}m/s")

## 2. Simulate Physics Validation Test

In a real environment, we would run the physics validation test using ROS 2 and Gazebo. Here we'll simulate the test results.

In [ ]:
# Simulate physics validation results
def simulate_fall_test(height, actual_gravity):
    """Simulate a test where we drop an object and measure fall time"""
    # Calculate expected time with actual gravity value
    expected_time = calculate_fall_time(height, actual_gravity)
    
    # Add small random variation to simulate real-world measurement
    import random
    variation = random.uniform(-0.005, 0.005)  # ±5ms variation
    measured_time = expected_time + variation
    
    # Calculate what gravity value would be inferred from measured time
    inferred_gravity = (2 * height) / (measured_time ** 2) * -1  # Negative for downward
    
    return measured_time, inferred_gravity

# Run simulated tests
test_height = 5.0
actual_gravity = -9.81  # What we expect in simulation

print(f"Running physics validation test...")
print(f"Test height: {test_height}m")
print(f"Simulated gravity: {actual_gravity} m/s²")

measured_time, inferred_gravity = simulate_fall_test(test_height, actual_gravity)
print(f"Expected fall time: {calculate_fall_time(test_height):.3f}s")
print(f"Measured fall time: {measured_time:.3f}s")
print(f"Inferred gravity: {inferred_gravity:.3f} m/s²")

# Validate results
gravity_error = abs(abs(inferred_gravity) - EXPECTED_GRAVITY)
if gravity_error <= GRAVITY_TOLERANCE:
    print(f"✅ GRAVITY VALIDATION PASSED: Within tolerance ({gravity_error:.3f} m/s² error)")
else:
    print(f"❌ GRAVITY VALIDATION FAILED: Outside tolerance ({gravity_error:.3f} m/s² error)")

## 3. Collision Validation

We also need to validate collision responses between objects.

In [ ]:
# Simulate collision validation
def simulate_collision_test(mass1, mass2, velocity1, velocity2=0, elasticity=0.8):
    """Simulate collision between two objects"""
    # Calculate momentum before collision
    momentum_before = mass1 * velocity1 + mass2 * velocity2
    
    # For elastic collision (with some energy loss based on elasticity)
    if mass2 > 0:  # Only calculate if there's a second object
        v1_final = ((mass1 - elasticity * mass2) * velocity1 + 
                    (1 + elasticity) * mass2 * velocity2) / (mass1 + mass2)
        v2_final = ((1 + elasticity) * mass1 * velocity1 + 
                    (mass2 - elasticity * mass1) * velocity2) / (mass1 + mass2)
    else:
        # If second object is stationary ground, simplify
        v1_final = -velocity1 * elasticity
        v2_final = 0
    
    momentum_after = mass1 * v1_final + mass2 * v2_final
    
    return v1_final, v2_final, momentum_before, momentum_after

# Run collision test
mass1, mass2 = 1.0, 2.0  # kg
vel1, vel2 = 3.0, 0.0  # m/s
elasticity = 0.7  # 70% elastic collision

v1_final, v2_final, p_before, p_after = simulate_collision_test(mass1, mass2, vel1, vel2, elasticity)
momentum_conservation_error = abs(p_before - p_after) / abs(p_before) * 100 if p_before != 0 else 0

print(f"Collision Test Results:")
print(f"  Object 1: {mass1}kg moving at {vel1}m/s")
print(f"  Object 2: {mass2}kg moving at {vel2}m/s")
print(f"  Elasticity: {elasticity}")
print(f"  After collision:")
print(f"    Object 1: {v1_final:.2f}m/s")
print(f"    Object 2: {v2_final:.2f}m/s")
print(f"  Momentum conservation error: {momentum_conservation_error:.2f}%")

if momentum_conservation_error < 5:  # Less than 5% error
    print(f"✅ COLLISION VALIDATION PASSED: Momentum conserved within tolerance")
else:
    print(f"❌ COLLISION VALIDATION FAILED: Momentum not conserved")

## 4. Visualization of Physics Results

Let's visualize how an object falls under the simulated gravity.

In [ ]:
# Plot object position over time under gravity
def calculate_position_over_time(fall_time, gravity=EXPECTED_GRAVITY, initial_height=5.0):
    """Calculate object position at different time points"""
    time_points = np.linspace(0, fall_time, 100)
    positions = initial_height + 0.5 * gravity * time_points**2  # s = ut + 1/2at^2 (u=0)
    return time_points, positions

# Calculate and plot
fall_time = calculate_fall_time(5.0)
time_vals, pos_vals = calculate_position_over_time(fall_time, actual_gravity, 5.0)

plt.figure(figsize=(10, 6))
plt.plot(time_vals, pos_vals, 'b-', linewidth=2, label='Object Position')
plt.xlabel('Time (seconds)')
plt.ylabel('Height (meters)')
plt.title('Object Fall Under Simulated Gravity (-9.81 m/s²)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--', label='Ground Level')
plt.legend()
plt.show()

# Also plot velocity over time
velocities = actual_gravity * time_vals  # v = u + at (u=0)

plt.figure(figsize=(10, 6))
plt.plot(time_vals, velocities, 'g-', linewidth=2, label='Object Velocity')
plt.xlabel('Time (seconds)')
plt.ylabel('Velocity (m/s)')
plt.title('Object Velocity Under Simulated Gravity')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--', label='Zero Velocity')
plt.legend()
plt.show()

## 5. Validation Summary

Run this section to get a complete summary of physics validation.

In [ ]:
# Summary of all validations
print("PHYSICS VALIDATION SUMMARY")
print("=" * 40)
print(f"Gravity Validation: {'PASSED' if gravity_error <= GRAVITY_TOLERANCE else 'FAILED'}")
print(f"Collision Validation: {'PASSED' if momentum_conservation_error < 5 else 'FAILED'}")
print(f"Expected gravity: {EXPECTED_GRAVITY} m/s²")
print(f"Simulated gravity: {actual_gravity} m/s²")
print(f"Gravity error: {gravity_error:.3f} m/s²")

overall_pass = (gravity_error <= GRAVITY_TOLERANCE) and (momentum_conservation_error < 5)
print(f"\nOverall Physics Validation: {'PASSED' if overall_pass else 'FAILED'}")

if overall_pass:
    print("\n✅ Physics simulation parameters are correctly configured!")
else:
    print("\n❌ Physics simulation needs adjustment.")
    if gravity_error > GRAVITY_TOLERANCE:
        print("  - Check gravity parameter in world file")
    if momentum_conservation_error >= 5:
        print("  - Check collision properties and physics engine settings")